<a href="https://colab.research.google.com/github/arnavjain2710/IITISoC24-ML-36/blob/main/My_own_fine_tuned_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from transformers import AutoFeatureExtractor, AutoModelForAudioClassification
import torch
import torch.nn.functional as F
from transformers import pipeline
import soundfile as sf

In [ ]:
! pip install sounddevice

In [ ]:
! pip install git+https://github.com/spatialaudio/python-sounddevice

  Cloning https://github.com/spatialaudio/python-sounddevice to /tmp/pip-req-build-ris_nacf
  Running command git clone --filter=blob:none --quiet https://github.com/spatialaudio/python-sounddevice /tmp/pip-req-build-ris_nacf
  Resolved https://github.com/spatialaudio/python-sounddevice to commit d5b872637f8c3f9eb41609e38ebc07e0726bb331
  Running command git submodule update --init --recursive -q
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!apt-get install -y portaudio19-dev
!pip install sounddevice


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libportaudio2 libportaudiocpp0
Suggested packages:
  portaudio19-doc
The following NEW packages will be installed:
  libportaudio2 libportaudiocpp0 portaudio19-dev
0 upgraded, 3 newly installed, 0 to remove and 45 not upgraded.
Need to get 188 kB of archives.
After this operation, 927 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudio2 amd64 19.6.0-1.1 [65.3 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudiocpp0 amd64 19.6.0-1.1 [16.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 portaudio19-dev amd64 19.6.0-1.1 [106 kB]
Fetched 188 kB in 1s (167 kB/s)
Selecting previously unselected package libportaudio2:amd64.
(Reading database ... 121913 files and directories currently installed.)
Preparing to unpack .../libportaudio2_19.6.0-1.

In [ ]:
! pip install pyaudio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 1.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for pyaudio (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pyaudio
Failed to build pyaudio
ERROR: Could not build wheels for pyaudio, which is required to install pyproject.toml-based projects


In [ ]:
!pip install ffmpeg-python

In [ ]:
from IPython.display import HTML, display, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import write as wav_write
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])

  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)

  riff_chunk_size = len(output) - 8
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  riff = output[:4] + bytes(b) + output[8:]
  sr, audio = wav_read(io.BytesIO(riff))

  # Save the audio to a file
  wav_write('output.wav', sr, audio)

  return audio, sr

# Run the function to record and save audio
audio, sr = get_audio()


In [ ]:
from transformers import AutoFeatureExtractor, AutoModelForAudioClassification
import torch
import torch.nn.functional as F
from transformers import pipeline
import soundfile as sf

In [ ]:
model_name= "VaidehiBhat007/fine_tuned_emo_classif"

In [ ]:
processor = AutoFeatureExtractor.from_pretrained(model_name)
model = AutoModelForAudioClassification.from_pretrained(model_name)

In [ ]:
import scipy.io.wavfile as wav
from scipy.signal import resample
import numpy as np

sampling_rate, data = wav.read("/content/output.wav")

new_sampling_rate = 16000

num_samples = int(len(data) * new_sampling_rate / sampling_rate)

resampled_data = resample(data, num_samples)

wav.write("resampled_audio.wav", new_sampling_rate, np.asarray(resampled_data, dtype=np.int16))


In [ ]:
audio_input, sampling_rate = sf.read("resampled_audio.wav")

inputs = processor(audio_input, sampling_rate=sampling_rate, return_tensors="pt")

In [ ]:
with torch.no_grad():
    outputs = model(**inputs)

In [ ]:
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
predicted_labels = predictions.argmax(dim=-1)

In [ ]:
print(predicted_labels)

tensor([5])


In [ ]:
label=predicted_labels.item()
if label==1:
  predicted_emotion="happy"
elif label==2:
  predicted_emotion="disgusted"
elif label==3:
  predicted_emotion="fearful"
elif label==4:
  predicted_emotion="sad"
elif label==5:
  predicted_emotion="angry"
elif label==0:
  predicted_emotion="neutral"
else:
  print("no emotion")

In [ ]:
print(f"Predicted emotion: {predicted_emotion}")


Predicted emotion: angry


In [ ]:
! pip install --upgrade pip
! pip install --upgrade git+https://github.com/huggingface/transformers.git accelerate


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.6 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-usfxfz92
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-usfxfz92
  Resolved https://github.com/huggingface/transformers.git to commit 7e1c7dc8b6989d96e7c35781bccb09468b49fc5e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 71.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
#from datasets import load_dataset


device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
)

'''dataset = load_dataset("distil-whisper/librispeech_long", "clean", split="validation")
sample = dataset[0]["audio"]

result = pipe(sample)
print(result["text"])'''


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


'dataset = load_dataset("distil-whisper/librispeech_long", "clean", split="validation")\nsample = dataset[0]["audio"]\n\nresult = pipe(sample)\nprint(result["text"])'

In [ ]:
sample1="/content/output.wav"

result1 = pipe(sample1)
print(result1)
print(result1["text"])

/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:480: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


{'text': ' I got job', 'chunks': [{'timestamp': (0.0, 3.18), 'text': ' I got job'}]}
 I got job


In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe1 = pipeline("text2text-generation", model="facebook/blenderbot-400M-distill")

In [ ]:
result2=pipe1("you are a conversational assistant, you need to keep the conversation going, given a text and an emotion associated with it your task is to reply to the text, your tone suitable as a reply for the user's emotion, also clearly mention the emotion given. text: "+ result1["text"]+ "emotion: " + predicted_emotion )

In [ ]:
print(result2)

[{'generated_text': ' I am sorry to hear that, I hope you can find a way to work through it.'}]


In [ ]:
print(result2[0]['generated_text'])

 That's great! I'm glad to hear that you are happy with your appointment!


In [ ]:
to_audio=result2[0]['generated_text']

In [ ]:

! pip install sentencepiece


In [ ]:
from transformers import pipeline
#from datasets import load_dataset
import soundfile as sf

synthesiser = pipeline("text-to-speech", "microsoft/speecht5_tts")

speaker_embedding = torch.randn(1, 512)
# You can replace this embedding with your own as well.

speech = synthesiser(to_audio, forward_params={"speaker_embeddings": speaker_embedding})

sf.write("speech.wav", speech["audio"], samplerate=speech["sampling_rate"])


In [ ]:
import librosa
import IPython.display as ipd
file_path = '/content/speech.wav'
speech, sr = librosa.load(file_path, sr=None)

# Display the audio player
ipd.Audio(speech, rate=sr)